![CoSAppLogo](images/cosapp.svg) **CoSApp** tutorials:

# Recorders

## What is a `Recorder`?!

A `Recorder` saves results of a [Driver](03-Drivers.ipynb) for data sharing and/or analysis.

## Introduction

### Add a `Recorder`

Simply use the `add_recorder` of a `Driver` method passing a recorder object you want to use 
(see [Available recorders](#Available-recorders) section of this tutorial).

In [ ]:
from cosapp.tests.library.systems import Multiply1
from cosapp.systems import System
from cosapp.drivers import RunOnce
from cosapp.recorders import DataFrameRecorder

m = Multiply1('mult')
run = m.add_driver(RunOnce('run', verbose=True))
run.add_recorder(DataFrameRecorder())

### Default behaviour

By default, units are appended to the variable name in headers. Special columns are also included:
- `Section`: User name reference
- `Reference`: Internal reference for the record
- `Status`: Status of the execution for the record
- `Error code`: Error code for the record

In [ ]:
m.run_drivers()
run.recorder.export_data()

### Common options

All recorders share the following options at creation:
- `includes`: variable name or list of variable names to be recorded. It supports traditional `*` and `?` wild cards characters.
- `excludes`: variable name or list of variable names not to be recorded. Also supports `*` and `?`. `excludes` shadow `includes`.
- `section`: section name of the recording (string)
- `hold`: boolean, determining whether or not the recorder should hold previous data between computations (`False` by default).
- `precision`: integer specifying the number of digits used when writing floating point number (9 by default).
- `raw_output`: boolean, `False` by default. If `True`, the recorder will not add units in variable headers, and will not add the special columns.

`section`, `hold` and `precision` can be modified after the recorder creation.


In [ ]:
class Example(System):
    
    def setup(self):
        self.add_inward('one', 0.)
        self.add_inward('one_two', [0., 1.])
        self.add_inward('two_1', 'test')
        self.add_inward('two_2', 42.)
        
e = Example('test')
run = e.add_driver(RunOnce('run'))
run.add_recorder(DataFrameRecorder(includes=['one*', 'two_?'], excludes=['one_*', ]))
e.run_drivers()

run.recorder.export_data()  # special method creating a pandas.DataFrame

In [ ]:
run.recorder.hold = True
e.one = 22.
e.run_drivers()
run.recorder.export_data()

## Available recorders

### DataFrameRecorder

Records data in a memory buffer, which can be exported as a `pandas.DataFrame` object after execution, using method `export_data()`.

### DSVRecorder

Records data in a Delimited Separated Value (DSV) formatted file. The constructor requires the file path, as a string.
It has two specific options:
- `delimiter`: delimiter symbol between columns (possibilities are `','`, `';'` or `'\t'`)
- `buffer`: boolean specifiying if the data must be written during the execution (`buffer=False`), or afterwards (`buffer=True`). Default is `False`.

In [ ]:
from cosapp.recorders import DSVRecorder
m = Multiply1('mult')
run = m.add_driver(RunOnce('run', verbose=True))
run.add_recorder(DSVRecorder('test.csv', delimiter=';', precision=2))
m.run_drivers()

In [ ]:
%less test.csv